# Requirements

In [280]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
import nltk
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense,LSTM
from tensorflow.keras import Sequential
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import regularizers

# Sample One hot encoding of word into vectors

In [281]:
sent=[  'the glass of milk',
     'the glass of juice',
     'the cup of tea',
    'I am a good boy',
     'I am a good developer',
     'understand the meaning of words',
     'your videos are good']


In [282]:
voc = 10000
word_vec = []
for words in sent:
    word_vec.append(one_hot(words,voc))
    

In [283]:
word_vec

[[1733, 8418, 7196, 2161],
 [1733, 8418, 7196, 9120],
 [1733, 3829, 7196, 52],
 [148, 1504, 8122, 7926, 6283],
 [148, 1504, 8122, 7926, 5027],
 [4688, 1733, 411, 7196, 6554],
 [930, 2709, 3247, 7926]]

## Embedding Layer - Padding the embedded word vec

In [284]:
padded_sent = pad_sequences(word_vec,padding='post',maxlen=len(sent))
padded_sent

array([[1733, 8418, 7196, 2161,    0,    0,    0],
       [1733, 8418, 7196, 9120,    0,    0,    0],
       [1733, 3829, 7196,   52,    0,    0,    0],
       [ 148, 1504, 8122, 7926, 6283,    0,    0],
       [ 148, 1504, 8122, 7926, 5027,    0,    0],
       [4688, 1733,  411, 7196, 6554,    0,    0],
       [ 930, 2709, 3247, 7926,    0,    0,    0]], dtype=int32)

## Embedding Layer - Real Embedding based on cosine similarity/context similarity

In [285]:
model = Sequential()    # creating the NN structure sequential
dim = 20            # dimension for the embedding layer
model.add(Embedding(input_dim=voc,output_dim=dim,input_length = len(sent)))     # embedding layer creation !!!
model.compile(optimizer='sgd',loss='mse')


In [286]:
model.predict(padded_sent)  # this is the embedded word vector from the Embedding Layer

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step


array([[[ 2.46940143e-02, -6.51972368e-03,  8.92732292e-03,
         -2.09624302e-02,  4.41537239e-02,  3.14967670e-02,
          1.62173249e-02,  4.75963689e-02, -1.97042357e-02,
         -3.06419376e-02,  1.65972598e-02, -4.94706631e-02,
         -3.18417326e-02, -2.52135154e-02, -3.90909426e-02,
          1.46145113e-02, -6.38295338e-03,  1.93407871e-02,
         -9.11815092e-03, -4.93485108e-02],
        [-4.88767885e-02, -9.41990688e-03,  4.96266373e-02,
          1.35069154e-02, -4.20153849e-02, -3.74471657e-02,
         -1.42593272e-02, -2.95222402e-02, -3.58071104e-02,
         -2.88829803e-02, -4.78815213e-02, -2.97435764e-02,
         -2.38282923e-02, -4.81404327e-02, -2.06377506e-02,
          1.76101588e-02, -8.19364935e-03, -2.63564233e-02,
          3.29629667e-02, -4.59516048e-03],
        [ 4.33257967e-03, -4.36495990e-04, -3.07105780e-02,
         -1.87085159e-02, -3.63255739e-02,  4.52335142e-02,
          4.42056991e-02,  3.67147587e-02, -1.21883973e-02,
         -3.

# IMDB dataset

In [287]:
data = pd.read_csv(r'E:\IMPORTANT STUFF\PYTHON\Notes\KRISHNAIK_GENAI_UDEMY\CODES\SIMPLE RNN\MYCODE\IMDB Dataset.csv')

In [288]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


# Splitting into Training and Testing datasets

In [289]:

X = data['review']
Y = data['sentiment']
le = LabelEncoder()
Y = le.fit_transform(Y)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=69)

In [290]:
X_train_list = X_train.tolist()
X_test_list = X_test.tolist()
Y_train_list = Y_train.tolist()
Y_test_list = Y_test.tolist()
Y_train_array = np.array(Y_train_list)
Y_test_array = np.array(Y_test_list)

# Tokenization - Preprocessing steps

In [ ]:
# tokenizing the datasets & Padding them
maxLen = 300
# X_train
tokenize = Tokenizer(num_words=10000)
tokenize.fit_on_texts(X_train_list)
tokenized_X_train = tokenize.texts_to_sequences(X_train_list)
padded_X_train = pad_sequences(tokenized_X_train,maxlen = maxLen,padding = 'post')


# X_test
tokenized_X_test = tokenize.texts_to_sequences(X_test_list)
padded_X_test = pad_sequences(tokenized_X_test,maxlen = maxLen,padding = 'post')



In [ ]:
print(np.unique(Y_train, return_counts=True))

(array([0, 1]), array([17527, 17473]))


# Developing the Model

In [ ]:
# building and training a simple RNN
model = Sequential()
model.add(Embedding(input_dim=10000,output_dim=150,input_length = maxLen))
model.add(LSTM(64, dropout=0.3, recurrent_dropout=0.3),)
model.add(Dense(1,activation='sigmoid',kernel_regularizer=regularizers.l2(0.01)))
model.compile(optimizer='adam',loss='binary_crossentropy')

e:\IMPORTANT STUFF\PYTHON\Notes\KRISHNAIK_GENAI_UDEMY\CODES\SIMPLE RNN\MYCODE\newProjEnv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Summary of the model
model.build(input_shape=(None, maxLen))
model.summary()

Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_24 (Embedding)        │ (None, 300, 150)       │     1,500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        55,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,555,105 (5.93 MB)

 Trainable params: 1,555,105 (5.93 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Creating a EarlyStopper for avoiding Overfitting
earlystopper = EarlyStopping(monitor='val_loss',patience=4,restore_best_weights=True)
model.fit(padded_X_train,Y_train_array,batch_size=30,epochs=30,callbacks=[earlystopper],validation_split=0.2)

Epoch 1/30
934/934 ━━━━━━━━━━━━━━━━━━━━ 103s 108ms/step - loss: 0.6915 - val_loss: 0.6604
Epoch 2/30
934/934 ━━━━━━━━━━━━━━━━━━━━ 101s 108ms/step - loss: 0.6615 - val_loss: 0.6525
Epoch 3/30
934/934 ━━━━━━━━━━━━━━━━━━━━ 101s 108ms/step - loss: 0.4964 - val_loss: 0.3398
Epoch 4/30
934/934 ━━━━━━━━━━━━━━━━━━━━ 101s 108ms/step - loss: 0.2795 - val_loss: 0.2931
Epoch 5/30
934/934 ━━━━━━━━━━━━━━━━━━━━ 101s 109ms/step - loss: 0.2110 - val_loss: 0.3020
Epoch 6/30
934/934 ━━━━━━━━━━━━━━━━━━━━ 102s 109ms/step - loss: 0.1683 - val_loss: 0.3051
Epoch 7/30
934/934 ━━━━━━━━━━━━━━━━━━━━ 102s 109ms/step - loss: 0.1351 - val_loss: 0.3384
Epoch 8/30
934/934 ━━━━━━━━━━━━━━━━━━━━ 101s 108ms/step - loss: 0.1094 - val_loss: 0.3539


In [ ]:
# Saving the model
model.save('modelLSTMRNN.h5')

# Model Prediction

In [ ]:
def sentimentAnalyze(sentence):
    tokenized_sentence = tokenize.texts_to_sequences([sentence.lower()])
    padded_sentence = pad_sequences(tokenized_sentence,maxlen = 300,padding = 'post')
    result = model.predict(padded_sentence)
    if result[0][0]>0.5:
        return ("Positive Comment!!!",result[0][0])
    else:
        return ('Negative Comment!!!',result[0][0])

print(sentimentAnalyze("Not my type!!"))